Day 1 (https://adventofcode.com/2023/day/1)

In [ ]:
import re

numbers_dict = {
    '0': '0',
    '1': '1',
    '2': '2',
    '3': '3',
    '4': '4',
    '5': '5',
    '6': '6',
    '7': '7',
    '8': '8',
    '9': '9',
    'zero': '0',
    'one': '1',
    'two': '2',
    'three': '3',
    'four': '4',
    'five': '5',
    'six': '6',
    'seven': '7',
    'eight': '8',
    'nine': '9'
}

# function for pt2 which transforms code with number-words into code with numbers
def transform_code(code):
    output = ['']*len(code)
    for k in numbers_dict:
        for x in re.finditer(k, code):
            output[x.start()] = numbers_dict[k]
    
    return ''.join(filter(lambda x: x, output))

with open('./inputs/day01.txt', 'r') as f:
    # read in the codes
    codes = list(map(lambda x: x.strip(), f.readlines()))

    # keep only the digits
    part1_codes = [''.join(filter(lambda x: x.isdigit(), c)) for c in codes]

    # keep only first and last digist and sum everything to get our answer
    print('Answer to Day 1, Part 1:', sum(map(int, [c[0] + c[-1] for c in part1_codes])))

    # converting word numbers into digits
    part2_codes = list(map(transform_code, codes))

    # keep only first and last digist and sum everything to get our answer
    print('Answer to Day 1, Part 2:', sum(map(int, [c[0] + c[-1] for c in part2_codes])))